## Exporting and Text Parsing of Drillhole Data
This notebook reads in from an export of the PostGIS/PostgreSQL WAMEX (Western Australian Mineral) database. A local subset copy instead of directly connecting to the database is used in this workshop. WAMEX contains mineral exploration reports and data that have lapsed the period of confidentiality. Online access is free of charge at:
https://www.dmp.wa.gov.au/Geological-Survey/Mineral-exploration-Reports-1401.aspx

From this database export, we pullout the following information: <br>
1 ) Collar (CollarID, Longitude, Latitude, RL, MaxDepth) <br>
2 ) Survey (CollarID, Depth, Azimuth, Dip) <br>
3 ) Lithology (CollarID, From_Depth, To_Depth, Company_LithoCode) <br>

The Survey table is further processed to calculate downhole X,Y,Z using minimum curvature.
The Lithology table is further processed to decode the Company LithoCodes into their respective lithologies. These lithologies are then standardized across different drilling campaigns and upscaled to be classified into broader rock types.

## WAMEX Database
Let's take a look at what the database information looks like and how these tables are related:
<a href="./graphics/gswadh_schema.pdf">gswadh_schema.pdf</a>
<img src='./graphics/gswadh_schema.bmp'>
<br>
Today, we will focus on tables necessary for collar, survey and lithology files.
<img src='./graphics/postgres-database.png'>

In [1]:
#import os, sys

#print(os.getcwd())
#sys.path.append(os.getcwd())

In [2]:
#sys.path

In [3]:
import pandas as pd
from dh2loop import dh2l

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
%run -i "dh2l_config.py"

Default parameters loaded from dh2l_config.py:


host_ = '130.95.198.59'
port_= 5432
DB_='gswa_dh'
user_ = 'postgres'
pwd_ = 'loopie123pgpw'
#export_path = ''


#File encoding type
encoding_1 ="utf-8"
encoding_2 = "ISO-8859-1"

#Database Files
data_path='../data/'
wamex_path='../data/wamex/'
collar_file=wamex_path+'collar.csv'
collarattr_file=wamex_path+'collarattr.csv'
dhsurvey_file=wamex_path+'dhsurvey.csv'
dhsurveyattr_file=wamex_path+'dhsurveyattr.csv'
dhgeology_file=wamex_path+'dhgeology.csv'
dhgeologyattr_file=wamex_path+'dhgeologyattr.csv'

#Thesauri
rl_maxdepth_dic_file= wamex_path+'rl_maxdepth_dic.csv'
survey_dic_file=wamex_path+'survey_dic.csv'
dic_attr_col_lithology_file=wamex_path+'dic_att_col_lithology.csv'
dic_attr_val_lithology_file=wamex_path+'dic_attr_val_lithology_filter.csv'
cleanup_lithology_file=wamex_path+'cleanup_lithology.csv'
litho_dic_file=wamex_path+'litho_dic_1.csv'
CET_hierarchy_dico_file=wamex_path+'hierarchy_dico.csv'

#ExportFiles
export_path='../data/ex

## Exporting Collar Data

Let's take a look at <b>collar</b> and <b>collarattr</b> table:

In [5]:
collar= pd.read_csv(collar_file,encoding = encoding_2, dtype='object')
collar.head()

,id,holeid,anumber,companyholeid,dataset,companyid,longitude,latitude,istransformed,modifieddate,modifiedby,mrtfileid,holetype,maxdepth,geom
0,153802,69189Minjar NorthSSRC007,69189,SSRC007,Minjar North,3049,116.949,-28.9058,True,NaN,NaN,5252,RC,127,0101000020E610000000000000C33C5D4000000000E0E7...
1,153803,69189Minjar NorthSSRC008,69189,SSRC008,Minjar North,3049,116.949,-28.9058,True,NaN,NaN,5252,RC,130,0101000020E6100000000000C0C23C5D4000000020E0E7...
2,153804,69189Minjar NorthSSRC009,69189,SSRC009,Minjar North,3049,116.949,-28.9058,True,NaN,NaN,5252,RC,120,0101000020E610000000000000C23C5D4000000040E0E7...
3,153813,69189Minjar NorthSSRC019,69189,SSRC019,Minjar North,3049,116.949,-28.9065,True,NaN,NaN,5252,RC,174,0101000020E610000000000040C03C5D40000000E00FE8...
4,153816,69189Minjar NorthSSRC022,69189,SSRC022,Minjar North,3049,116.949,-28.9068,True,NaN,NaN,5252,RC,146,0101000020E610000000000080C23C5D400000004027E8...


In [6]:
collarattr= pd.read_csv(collarattr_file,encoding = encoding_2, dtype='object')
collarattr.loc[collarattr['collarid'] == '153802']

,id,collarid,attributecolumn,attributevalue,modifieddate,modifiedby,mrtdetailid
0,1769961,153802,Elevation,359.251,NaN,NaN,116852
1,1769962,153802,TD,127,NaN,NaN,116853
2,1769963,153802,Drilling_code,RC,NaN,NaN,116854
3,1769964,153802,Tenement,M59/421,NaN,NaN,116855
4,1769965,153802,Drill_Date,30-Nov-03,NaN,NaN,116856
6,1769966,153802,Drilling_Company,"""Layne Drilling Pty. Ltd., Perth""",NaN,NaN,116857
7,1769967,153802,HoleDiam,135mm,NaN,NaN,116858
8,1769968,153802,PSI,300,NaN,NaN,116859
9,1769969,153802,Prospect,Silverstone,NaN,NaN,116861


We can see that for drillhole# 153802: <br>
RL = Elevation = 359.251 <br>
MaxDepth = TD =127<br>

Now, let's look at another hole (# 792833):

In [7]:
collarattr.loc[collarattr['collarid'] == '792942']

,id,collarid,attributecolumn,attributevalue,modifieddate,modifiedby,mrtdetailid
141,9597660,792942,RL,366.147,NaN,NaN,511733
142,9597661,792942,Depth (m),160,NaN,NaN,511734
143,9597662,792942,MGA Azimuth,317.80,NaN,NaN,511735
144,9597663,792942,Dip,-59.59,NaN,NaN,511736
145,9597664,792942,Drill Type,RC,NaN,NaN,511737
146,9597665,792942,Grid,"""GDA94, MGA50""",NaN,NaN,511738
147,9597666,792942,Drilling Company,"""Strange Drilling, Kalgoorlie""",NaN,NaN,511739
148,9597667,792942,Date Completed,13/05/06,NaN,NaN,511740


We can observe that this time: <br>
RL = RL = 366.147 <br>
MaxDepth = Depth (m) =160<br>    

In order to pullout the corresponding RL and MaxDepth values for each drillhole, it was necessary to build a thesaurus.
Here is a quick look into the <b>RL and MaxDepth thesaurus</b>:

In [8]:
rl_maxdepth_dic= pd.read_csv(rl_maxdepth_dic_file,encoding = encoding_2, dtype='object')
rl_maxdepth_dic.loc[rl_maxdepth_dic['cet_attributecolumn'] == 'rl']

,attributecolumn,cet_attributecolumn
111,"""""""RL""""""",rl
112,"""Nominal\r\nRL """,rl
113,"""RLLocal""",rl
114,"""RLOrig""",rl
115,"""RLSurvMethod""",rl
...,...,...
505,SUR DEPTH,rl
506,RL_Method,rl
507,Depth_Start_m,rl
509,ELEVATION,rl


In [9]:
rl_maxdepth_dic.loc[rl_maxdepth_dic['cet_attributecolumn'] == 'maxdepth']

,attributecolumn,cet_attributecolumn
0,TD,maxdepth
1,Total depth,maxdepth
2,"""""""DEPTH""""""",maxdepth
3,"""""""TOT_DEPTH""""""",maxdepth
4,"""Final\r\nDepth(m)""",maxdepth
...,...,...
488,Survey Depth,maxdepth
489,Sur_Depth,maxdepth
490,EOH_depth,maxdepth
491,Precollar Depth,maxdepth


Using the thesaurus and a little bit of code and cleaning up, we can extract the collar information.
    1. CollarID
    2. HoleId
    3. Longitude
    4. Latitude
    5. RL
    6. MaxDepth
    7. X
    8. Y

In [10]:
dh2l.collar_attr_col_dic(rl_maxdepth_dic_file)
dh2l.collar_collarattr_final(collar_file, collarattr_file, rl_maxdepth_dic_file, DB_Collar_Export)
DB_Collar_Export_table=pd.read_csv(DB_Collar_Export,encoding = encoding_2, dtype='object')
DB_Collar_Export_table= DB_Collar_Export_table.loc[:, ~DB_Collar_Export_table.columns.str.contains('^Unnamed')]
print(DB_Collar_Export_table)

    CollarID                    HoleId Longitude  Latitude       RL MaxDepth
0    1146526  71288Minjar NorthSSDD006   116.948  -28.8991  356.255    546.4
1    1146528  71288Minjar NorthSSDD008   116.947  -28.9021  356.871    405.0
2    1146530  71288Minjar NorthSSDD010   116.949  -28.9083  360.214    249.0
3    1146531  71288Minjar NorthSSDD011   116.948  -28.9002  357.153    468.0
4    1146650  71288Minjar NorthSSDD005   116.948  -28.9002  357.156      150
..       ...                       ...       ...       ...      ...      ...
127   931215         79530KararaLSC015   117.151  -29.4116  315.535      124
128   931216         79530KararaLSC016   117.148   -29.426  361.603      158
129   931217         79530KararaLSC017   117.144  -29.4045  323.052      136
130   931218         79530KararaLSC018   117.151  -29.4335  365.334      248
131   931219         79530KararaLSD006   117.153  -29.4336  368.845    260.7

[132 rows x 6 columns]


In [11]:
dh2l.plot_collar (DB_Collar_Export, geology)

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\plotting.py:471: UserWarning: Only specify one of 'column' or 'color'. Using 'color'.
  "Only specify one of 'column' or 'color'. Using 'color'.", UserWarning


Let's also calculate for the projected coordinates:

In [12]:
DB_Collar_Export_table = DB_Collar_Export_table.loc[:, ~DB_Collar_Export_table.columns.str.contains('^Unnamed')]
DB_Collar_Export_table[['X','Y']] = DB_Collar_Export_table.apply(dh2l.convert_coords,axis=1)
DB_Collar_Export_table.to_csv(DB_Collar_Export)
print(DB_Collar_Export_table)

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

...

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\00103098\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization

    CollarID                    HoleId Longitude  Latitude       RL MaxDepth  \
0    1146526  71288Minjar NorthSSDD006   116.948  -28.8991  356.255    546.4   
1    1146528  71288Minjar NorthSSDD008   116.947  -28.9021  356.871    405.0   
2    1146530  71288Minjar NorthSSDD010   116.949  -28.9083  360.214    249.0   
3    1146531  71288Minjar NorthSSDD011   116.948  -28.9002  357.153    468.0   
4    1146650  71288Minjar NorthSSDD005   116.948  -28.9002  357.156      150   
..       ...                       ...       ...       ...      ...      ...   
127   931215         79530KararaLSC015   117.151  -29.4116  315.535      124   
128   931216         79530KararaLSC016   117.148   -29.426  361.603      158   
129   931217         79530KararaLSC017   117.144  -29.4045  323.052      136   
130   931218         79530KararaLSC018   117.151  -29.4335  365.334      248   
131   931219         79530KararaLSD006   117.153  -29.4336  368.845    260.7   

                 X             Y  
0   

## Exporting Survey Data

 The same approach is done for survey data. Let's take a look at <b>dhsurvey</b> and <b>dhsurveyattr</b>.

In [13]:
dhsurvey= pd.read_csv(dhsurvey_file,encoding = encoding_2, dtype='object')
#dhsurvey.head()
dhsurvey.loc[dhsurvey['id'] == '318088']

,id,collarid,depth,units,accuracy,loaddate,loadby,modifieddate,modifiedby,mrtfileid,dip,azimuth
1,318088,153802,120,NaN,NaN,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,5256,NaN,NaN


In [14]:
dhsurveyattr= pd.read_csv(dhsurveyattr_file,encoding = encoding_2, dtype='object')
dhsurveyattr.loc[dhsurveyattr['dhsurveyid'] == '318088']

,id,dhsurveyid,attributecolumn,attributevalue,loaddate,loadby,modifieddate,modifiedby,mrtdetailid
5,1855500,318088,Dip,-52,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,116918
6,1855501,318088,Azimuth,90,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,116919
7,1855502,318088,Azimuth,87,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,116920
8,1855503,318088,Instrument,Eastman,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,116922
9,1855504,318088,Prospect,Silverstone,2011-07-18 09:34:26.913,MIGPSMB,NaN,NaN,116923


This time we use an <b> Azimuth_Dip Thesaurus </b>.

In [15]:
survey_dic= pd.read_csv(survey_dic_file,encoding = encoding_2, dtype='object')
survey_dic.loc[survey_dic['new_name'] == 'AZI']

,attr_col,new_name
0,Aximuth,AZI
1,AZ,AZI
2,AZ_COMET,AZI
3,AZ_Eelya,AZI
4,AZ_MGX_LOC,AZI
5,AZ_Webbs,AZI
6,AZIM,AZI
7,Azim,AZI
8,AZIMUTH,AZI
9,Azimuth,AZI


In [16]:
survey_dic.loc[survey_dic['new_name'] == 'DIP']

,attr_col,new_name
142,Dip,DIP
143,DIP,DIP
144,dip,DIP
145,Dip (deg.),DIP
146,Dip_2,DIP
147,INC,DIP
148,Inclination,DIP
149,DIP (deg.),DIP


In [17]:
dh2l.survey_attr_col_dic(survey_dic_file)
dh2l.survey_final(dhsurvey_file,dhsurveyattr_file, DB_Survey_Export)
dhsurvey= pd.read_csv(DB_Survey_Export,encoding = encoding_2, dtype='object')
dhsurvey= dhsurvey.loc[:, ~dhsurvey.columns.str.contains('^Unnamed')]
print(dhsurvey)

     CollarID  Depth Azimuth    Dip
0      153802    0.0    90.0  -50.0
1      153802  120.0    90.0  -90.0
2      153803    0.0    90.0  -55.0
3      153803  130.0    90.0  -90.0
4      153804    0.0    90.0  -60.0
...       ...    ...     ...    ...
1346  1233228  180.0   137.2  -90.0
1347  1233228  200.0   137.3  -90.0
1348  1233228  220.0   137.4  -90.0
1349  1233228  230.0   137.4  -90.0
1350  1233229    0.0   135.0  -60.0

[1351 rows x 4 columns]


Let's also calculate the downhole xyz from the depth, azimuth and dip values using the Minimum Curvature Method. It smooths two straight-line segments of the Balanced Tangential Method by using the Ratio Factor (RF = 2*tan(dl/2)/dl).

<img src='./graphics/Minimum-Curvature-Method.jpg'>

Where:  
MD = Measured Depth between surveys  
I1 = Inclination (angle) of upper survey in degrees  
I2 = Inclination (angle) of lower in degrees  
Az1= Azimuth direction of upper survey  
Az2 = Azimuth direction of lower survey  
RF = Ratio Factor  
ß  = the dog leg angle.  

Sources:
http://www.drillingformulas.com/minimum-curvature-method/ <br>
https://gis.stackexchange.com/questions/13484/how-to-convert-distance-azimuth-dip-to-xyz <br>
http://www.cgg.com/data//1/rec_docs/2269_MinimumCurvatureWellPaths.pdf`<br>

In [18]:
dh2l.convert_survey(DB_Collar_Export,DB_Survey_Export, DB_Survey_Export_Calc)
dhsurvey_calc= pd.read_csv(DB_Survey_Export_Calc,encoding = encoding_2, dtype='object')
dhsurvey_calc.dropna()
dhsurvey_calc= dhsurvey_calc.loc[:, ~dhsurvey_calc.columns.str.contains('^Unnamed')]
print(dhsurvey_calc)

     CollarID       Depth     Azimuth         Dip Index              X  \
0      153802    0.000000   90.000000  -50.000000     2  495028.106029   
1      153802    0.000000   90.000000  -50.000000     2  494917.618977   
2      153803    0.000000   90.000000  -55.000000     4  495028.106029   
3      153803    0.000000   90.000000  -55.000000     4  494906.041567   
4      153804    0.000000   90.000000  -60.000000     6  495028.106029   
...       ...         ...         ...         ...   ...            ...   
1345  1233228  140.000000  137.400000  -90.000000  1339  482677.460027   
1346  1233228  160.000000  137.800000  -90.000000  1339  482663.948285   
1347  1233228  180.000000  137.200000  -90.000000  1339  482650.372272   
1348  1233228  200.000000  137.300000  -90.000000  1339  482636.821912   
1349  1233228  220.000000  137.400000  -90.000000  1339  482630.053152   

                   Y           Z  
0     6802449.793954  359.251000  
1     6802449.793954  319.037002  
2     

The survey data export includes the following data:
    1. CollarID
    2. Depth
    3. Azimuth
    4. Dip
    5. X
    6. Y
    7. Z

## Exporting Lithology Data

Again, we have a pair of tables: <b> dhgeology </b> and <b> dhgeologyattr</b>.

In [19]:
dhgeology= pd.read_csv(dhgeology_file,encoding = encoding_2, dtype='object')
dhgeology.head()

,id,collarid,fromdepth,todepth,units,accuracy,modifieddate,modifiedby,mrtfileid
0,2537823,153803,59,61,NaN,NaN,NaN,NaN,5255
1,2537824,153803,61,62,NaN,NaN,NaN,NaN,5255
2,2537825,153803,62,69,NaN,NaN,NaN,NaN,5255
3,2537826,153803,69,72,NaN,NaN,NaN,NaN,5255
4,2537827,153803,72,73,NaN,NaN,NaN,NaN,5255


In [20]:
dhgeologyattr= pd.read_csv(dhgeologyattr_file,encoding = encoding_2, dtype='object')
dhgeologyattr.head()

,id,dhgeologyid,attributecolumn,attributevalue,modifieddate,modifiedby,mrtdetailid
0,20532422,2537765,Weathering_Intensity,ew,NaN,NaN,116894
1,20532423,2537765,Regolith,Lf,NaN,NaN,116895
2,20532424,2537765,Lithology1,Czlu,NaN,NaN,116896
3,20532425,2537765,Grain_Size,vfg,NaN,NaN,116898
4,20532426,2537765,Colour,bnrd,NaN,NaN,116899


In [21]:
dh2l.litho_attr_val_dic(dic_attr_val_lithology_file)
dh2l.litho_dico (litho_dic_file)
dh2l.clean_up (cleanup_lithology_file)
dh2l.litho_attr_val_with_fuzzy (CET_Litho)

In [22]:
dh2l.litho_final (collar_file, dhgeology_file, dhgeologyattr_file, dic_attr_col_lithology_file, CET_Litho, DB_Lithology_Export)

Exporting lithology is a bit more tricky than collar and survey because what we get are rock codes (i.e., BAS, GRA) that have to be further decoded.
Let's breakdown the steps done in the export.

<img src='./graphics/lithology_thesauri.png'>

First, the Company_LithoCodes are pulled out the same way we did for Collar and Survey. We use a <b> Litho thesaurus (Thesuarus 1) </b> that lists all the attribute columns that refer to lithology.

In [23]:
dic_att_col_lithology=pd.read_csv(dic_attr_col_lithology_file,encoding = encoding_2, dtype='object')
print(dic_att_col_lithology)

     attributecolumn
0       lithological
1       lihological 
2                LIT
3               LITH
4               Lith
5             Lith_1
6             LITH_1
7        Lith_1_Code
8          Lith_code
9          lith_code
10         Lith_Code
11        lith_code1
12        Lith_Code1
13    Lith_Qualifier
14  Lith_Quallifier1
15             Lith1
16             LITH1
17         Lith1_100
18         Lith1_101
19         Lith1_Cat
20        LITH1_CODE
21        lith1_code
22        Lith1_code
23        Lith1_Code
24         Lith1Code
25         LITHOLOGY
26         lithology
27         Lithology
28      LITHOLOGY  1
29       Lithology 1
30       Lithology_1
31        LITHOLOGY1
32        Lithology1
33    Main Geol Unit
34    Maj. Lithology
35        Major Rock
36              RCK1
37              rck1
38              Rock
39              ROCK
40            Rock 1
41             Rock1
42             ROCK1
43             rock1
44          ROCKCODE
45          Rockcode
46         Ro

The <b> second thesaurus (from logging protocols) </b> decodes the Company_Litho for the corresponding Company_LithoCode.

In [24]:
dic_attr_val_lithology=pd.read_csv(dic_attr_val_lithology_file,encoding = encoding_2, dtype='object')
dic_attr_val_lithology.head()

,company_id,company_code,comapany_litho
0,11410,GR,Granite
1,11410,MG,Gabbro
2,11410,SA,Undifferentiated arenite
3,11410,CL,Clay
4,12411,Gr,Granitiod undifferentiated


Before performing the fuzzywuzzy matching, the Company_Litho text is cleaned to remove extra symbols and descriptions. This <b> cleanup_dictionary </b> includes mineral names, ages and qualifiers (i.e, dominant, abundant, minor).

In [25]:
cleanup_lithology=pd.read_csv(cleanup_lithology_file,encoding = encoding_2, dtype='object')
print(cleanup_lithology)

          clean
0      abundant
1    actinolite
2    adcumulate
3      adjacent
4      adularia
..          ...
577     younger
578     zeolite
579        zinc
580      zircon
581     zoisite

[582 rows x 1 columns]


We use the fuzzywuzzy algorithm to standardize the lithologies. The FuzzyWuzzy matching compares the <b> Company_Litho </b> phrase against a <b> lithology thesaurus </b>. FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. It uses the Levenshtein Distance to calculate the differences between sequences.

Let's take a look at the <b> Lithology Thesaurus (Thesaurus 3) </b>. Each row refers to one entry. Each synonym is separated by a space.

In [26]:
lithology_thesaurus=pd.read_csv(litho_dic_file,encoding = encoding_2, dtype='object')
lithology_thesaurus[lithology_thesaurus['clean'].str.contains('Banded_iron_formation')]

,clean
244,Banded_iron_formation iron-formationBIF Banded...


The processor we used to do the matching is: <b>fuzz.token_set_ratio </b>. It tokenize strings, but split the tokens into groups: intersection and remainder before comparing. It is not as strict as a using an exact processor but stricter than a partial match. Having a comprehensive thesaurus allows us this processor to work effectively. For more information on this: https://www.geeksforgeeks.org/fuzzywuzzy-python-library/

Here's the pseudocode:<br>
&emsp; scores=process.extract(Cleaned_Company_Litho, Litho_Dico, scorer=fuzz.token_set_ratio)<br>
&emsp; &emsp; for sc in scores:                     
&emsp; &emsp; &emsp; if(sc[score]>bestmatch): #better than previous best match<br>
&emsp; &emsp; &emsp; &emsp; bestmatch =  sc[score]<br>
&emsp; &emsp; &emsp; &emsp; bestlitho= Litho_Dico[firstword]<br>
&emsp; &emsp; &emsp; &emsp; if(sc[litho]==Cleaned_Company_Litho[last]): #bonus for being last word in phrase<br>
&emsp; &emsp; &emsp; &emsp; bestmatch=bestmatch*1.01<br>
&emsp; &emsp; &emsp; elif (sc[score]==bestmatch): #equal to previous best match<br>
&emsp; &emsp; &emsp; &emsp; if(sc[0]==words[last]): #bonus for being last word in phrase<br>
&emsp; &emsp; &emsp; &emsp; &emsp; bestlitho=Litho_Dico[firstword]<br>
&emsp; &emsp; &emsp; &emsp; &emsp; bestmatch=bestmatch*1.01<br>
<br>

The pseudocode shows that we input the <b> cleaned Company_Litho </b> and we parse it through a <b> lithology thesaurus </b>. We take in the score for each iteration and if it is greater than the previous match, it stores the <b> score (bestmatch) </b> and the <b> first lithology (CET_Litho) </b> listed in the corresponding lithology thesaurus entry. A bonus is also added to the score if the cleaned Company_Litho's last word matches the thesaurus.
<br>
Furthermore, if the match is less than a threshold we set (in this case, 80). The <b> CET_Litho </b> is classified as "unclassified _rock".

Let's take a look at the results:

In [27]:
DB_Lithology_Export_View=pd.read_csv(DB_Lithology_Export,encoding = encoding_2, dtype='object')
print(DB_Lithology_Export_View)

     Company_ID CollarID FromDepth ToDepth Company_LithoCode  \
0          3050  1146526        23      31                 X   
1          3050  1146526        23      31                 X   
2          3050  1146526        31      32                 X   
3          3050  1146526        31      32                 X   
4          3050  1146526        32      38                 X   
...         ...      ...       ...     ...               ...   
4344       3050   931220     193.3   212.3               Sbl   
4345       3050   931220     212.3   213.6               Ssh   
4346       3050   931220     213.6   214.8               Sbl   
4347       3050   931220     214.8  217.88                 $   
4348       3050   931220    217.88   229.4                 $   

               Company_Litho          CET_Litho Score Unnamed: 8  
0     breccia origin unknown  unclassified_rock   101        NaN  
1               unknown rock               rock   101        NaN  
2     breccia origin unknown  

## Upscaling Lithology Data

In order to link and upscale the drillhole information, a <b> hierarchical thesaurus (Thesaurus 4) </b> was also built. The dictionary involved three levels that would upscale a list of 757 rock names to more general rock groups.

In [28]:
CET_hierarchy_dico = pd.read_csv(CET_hierarchy_dico_file,encoding = encoding_2)

In [29]:
dh2l.upscale_litho (DB_Lithology_Export, CET_hierarchy_dico_file, DB_Lithology_Upscaled)
Upscaled_Litho= pd.read_csv(DB_Lithology_Upscaled,encoding = encoding_2)
Upscaled_Litho.head()

,Unnamed: 0,Company_ID,CollarID,FromDepth,ToDepth,Company_LithoCode,Company_Litho,CET_Litho,Score,Level_3,Level_2,Level_1
0,0,3050,1146526,23.0,31.0,X,unknown rock,rock,101,rock,natural_superficial_ground,surficial
1,1,3050,1146526,31.0,32.0,X,unknown rock,rock,101,rock,natural_superficial_ground,surficial
2,2,3050,1146526,32.0,38.0,X,unknown rock,rock,101,rock,natural_superficial_ground,surficial
3,3,3050,1146526,38.0,39.0,X,unknown rock,rock,101,rock,natural_superficial_ground,surficial
4,4,3050,1146526,39.0,42.0,X,unknown rock,rock,101,rock,natural_superficial_ground,surficial


In [30]:
Upscaled_Litho[Upscaled_Litho['Level_1'] == "igneous"]

,Unnamed: 0,Company_ID,CollarID,FromDepth,ToDepth,Company_LithoCode,Company_Litho,CET_Litho,Score,Level_3,Level_2,Level_1
35,35,3050,1146526,352.8,356.1,As,syenite,syenite,101,syenite,intermediate_coarse-grained crystalline,igneous
36,36,3050,1146530,193.3,197.7,As,syenite,syenite,101,syenite,intermediate_coarse-grained crystalline,igneous
37,37,3050,1146528,76.0,78.0,Ac,carbonatite,carbonatite,101,carbonatite,exotic_crystalline,igneous
38,38,3050,1146528,78.0,80.0,Ac,carbonatite,carbonatite,101,carbonatite,exotic_crystalline,igneous
39,39,3050,1146528,80.0,84.0,Ac,carbonatite,carbonatite,101,carbonatite,exotic_crystalline,igneous
...,...,...,...,...,...,...,...,...,...,...,...,...
3885,3885,3050,931217,29.0,60.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3886,3886,3050,931217,60.0,64.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3887,3887,3050,931217,65.0,66.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3890,3890,3050,548917,94.0,103.0,Mg,gabbro,gabbro,101,gabbro,mafic_coarse-grained crystalline,igneous


In [31]:
Upscaled_Litho[Upscaled_Litho['Level_2'] == "ore"]

,Unnamed: 0,Company_ID,CollarID,FromDepth,ToDepth,Company_LithoCode,Company_Litho,CET_Litho,Score,Level_3,Level_2,Level_1
163,163,1621,1233095,0.00,12.00,Sif,iron-formation,banded_iron_formation,90,banded_iron_formation,ore,mineralisation
164,164,1621,1233095,12.00,17.00,Sif,iron-formation,banded_iron_formation,90,banded_iron_formation,ore,mineralisation
165,165,1621,1233095,17.00,29.00,Sif,iron-formation,banded_iron_formation,90,banded_iron_formation,ore,mineralisation
166,166,1621,1233095,29.00,35.00,Sif,iron-formation,banded_iron_formation,90,banded_iron_formation,ore,mineralisation
167,167,1621,1233095,35.00,42.00,Sif,iron-formation,banded_iron_formation,90,banded_iron_formation,ore,mineralisation
...,...,...,...,...,...,...,...,...,...,...,...,...
3753,3753,3050,931213,183.00,185.00,S,massive sulphides,massive_sulphide,91,massive_sulphide,ore,mineralisation
3754,3754,3050,931213,185.00,186.00,S,massive sulphides,massive_sulphide,91,massive_sulphide,ore,mineralisation
3755,3755,3050,931213,186.00,190.00,S,massive sulphides,massive_sulphide,91,massive_sulphide,ore,mineralisation
3756,3756,3050,931220,214.80,217.88,$,massive sulphides,massive_sulphide,91,massive_sulphide,ore,mineralisation


In [32]:
Upscaled_Litho[Upscaled_Litho['CET_Litho'] == "granodiorite"]

,Unnamed: 0,Company_ID,CollarID,FromDepth,ToDepth,Company_LithoCode,Company_Litho,CET_Litho,Score,Level_3,Level_2,Level_1
3867,3867,3050,548917,0.0,19.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3868,3868,3050,548917,19.0,27.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3869,3869,3050,548917,27.0,31.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3870,3870,3050,548917,31.0,39.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3871,3871,3050,548917,39.0,41.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3872,3872,3050,548917,41.0,50.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3873,3873,3050,548917,54.0,55.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3874,3874,3050,548917,55.0,60.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3875,3875,3050,548917,65.0,73.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous
3876,3876,3050,548917,73.0,87.0,Fgd,granodiorite,granodiorite,101,granodiorite,felsic_coarse-grained crystalline,igneous


Let's also calculate for the X,Y,Z coordinates for FromDepth, ToDepth and its midpoint.

In [33]:
dh2l.convert_lithology(DB_Collar_Export, DB_Survey_Export, DB_Lithology_Upscaled, DB_Lithology_Export_Calc)
DB_Lithology_Export_Calculated=pd.read_csv(DB_Lithology_Export_Calc)
print(DB_Lithology_Export_Calculated)

      Unnamed: 0  Company_ID   CollarID  FromDepth  ToDepth Company_LithoCode  \
0             36        3050   548917.0       0.00    19.00               Fgd   
1             37        3050   548917.0      19.00    27.00               Fgd   
2             38        3050   548917.0      27.00    31.00               Fgd   
3             39        3050   548917.0      31.00    39.00               Fgd   
4             40        3050   548917.0      39.00    41.00               Fgd   
...          ...         ...        ...        ...      ...               ...   
3229        3333        1621  1233228.0     190.50   193.20               Ssh   
3230        3334        1621  1233228.0     193.20   229.44               Sif   
3231        3335        1621  1233228.0     229.44   231.13               Sif   
3232        3336        1621  1233228.0     231.13   233.25               Ssh   
3233        3337        1621  1233228.0     233.25   245.00               Ssh   

       Company_Litho       

## Export to VTK

In [34]:
dh2l.intervals2vtk(DB_Lithology_Export_Calc, DB_Lithology_Export_VTK)

<img src='./graphics/vtk.png'>

Open Paraview to view the VTK files and have some fun!<br>
The VTK file output is at \dh2loop\data\export\DB_Lithology_Export.vtp

In [35]:
#import pyvista
#import pandas as pd
#import numpy as np
#from PVGeo.filters import PointsToTube

#table= pd.read_csv('../data/export/DB_Lithology_Export_Calc.csv',encoding = "ISO-8859-1", dtype='object')
#xb = table['xmt'].values.astype(float)
#yb = table['ymt'].values.astype(float)
#zb = table['zmt'].values.astype(float)

#points = pyvista.PolyData(np.c_[xb,yb,zb])
#points.plot(point_size=10)
#tube = PointsToTube(nearestNbr=True).apply(points)
#tube.plot(color=True)